# Sentanal
A sentiment analysis classifier.

In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv("data.csv")
data.sample()

,passage,label
7,"If you go to this museum, you're going to have...",0
